 # Análisis exploratorio de datos:

## 1. Importar librerías

In [1]:
import pandas as pd
import numpy as np
from scipy import stats as st
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.io as pio

## 2. Importar archivos

In [2]:
cabs=pd.read_csv('datasets/moved_project_sql_result_01.csv')
trips=pd.read_csv('datasets/moved_project_sql_result_04.csv')
loop_trips=pd.read_csv('datasets/moved_project_sql_result_07.csv')

## 3. Estudio de los datos de los ficheros

### 3.1 Tipos de datos

In [3]:
cabs.head(5)


,company_name,trips_amount
0,Flash Cab,19558
1,Taxi Affiliation Services,11422
2,Medallion Leasin,10367
3,Yellow Cab,9888
4,Taxi Affiliation Service Yellow,9299


In [4]:
trips.head(5)


,dropoff_location_name,average_trips
0,Loop,10727.466667
1,River North,9523.666667
2,Streeterville,6664.666667
3,West Loop,5163.666667
4,O'Hare,2546.900000


In [5]:
loop_trips.head(5)

,start_ts,weather_conditions,duration_seconds
0,2017-11-25 16:00:00,Good,2410.0
1,2017-11-25 14:00:00,Good,1920.0
2,2017-11-25 12:00:00,Good,1543.0
3,2017-11-04 10:00:00,Good,2512.0
4,2017-11-11 07:00:00,Good,1440.0


In [6]:
# Revisamos los tipos de datos de los dataframes
cabs.info()
trips.info()
loop_trips.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 64 entries, 0 to 63
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   company_name  64 non-null     object
 1   trips_amount  64 non-null     int64 
dtypes: int64(1), object(1)
memory usage: 1.1+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 94 entries, 0 to 93
Data columns (total 2 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   dropoff_location_name  94 non-null     object 
 1   average_trips          94 non-null     float64
dtypes: float64(1), object(1)
memory usage: 1.6+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1068 entries, 0 to 1067
Data columns (total 3 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   start_ts            1068 non-null   object 
 1   weather_conditions  1068 non-null   object 
 2   duration_se

Los datos de fecha están como strings, asi que lo pasaremos a tipo datetime.

In [7]:
#Cambiamos el tipo de dato a datetime
loop_trips['start_ts']=pd.to_datetime(loop_trips['start_ts'])
loop_trips.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1068 entries, 0 to 1067
Data columns (total 3 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   start_ts            1068 non-null   datetime64[ns]
 1   weather_conditions  1068 non-null   object        
 2   duration_seconds    1068 non-null   float64       
dtypes: datetime64[ns](1), float64(1), object(1)
memory usage: 25.2+ KB


### 3.2 Tratamiento de ausentes y duplicados

In [8]:
#Calculamos los ausentes
print('Ausentes cabs:\n',cabs.isna().sum())
print('Ausentes trips:\n',trips.isna().sum())
print('Ausentes loop_trips:\n',loop_trips.isna().sum())      

Ausentes cabs:
 company_name    0
trips_amount    0
dtype: int64
Ausentes trips:
 dropoff_location_name    0
average_trips            0
dtype: int64
Ausentes loop_trips:
 start_ts              0
weather_conditions    0
duration_seconds      0
dtype: int64


In [9]:
#Calculamos los duplicados
print('Duplicados cabs:\n',cabs.duplicated().sum())
print('Duplicados trips:\n',trips.duplicated().sum())
print('Duplicados loop_trips:\n',loop_trips.duplicated().sum()) 

Duplicados cabs:
 0
Duplicados trips:
 0
Duplicados loop_trips:
 197


In [10]:
#Revisamos los duplicados
print(loop_trips[loop_trips.duplicated()].head(10))
print()
#Calculamos el porcentaje de duplicados
print('Porcentaje de duplicados: ',100*loop_trips.duplicated().sum()/loop_trips.shape[0])

               start_ts weather_conditions  duration_seconds
62  2017-11-11 06:00:00               Good            1260.0
74  2017-11-11 08:00:00               Good            1380.0
76  2017-11-04 09:00:00               Good            1380.0
117 2017-11-11 07:00:00               Good            1380.0
119 2017-11-04 14:00:00               Good            3300.0
125 2017-11-11 08:00:00               Good            1380.0
126 2017-11-11 09:00:00               Good            1380.0
130 2017-11-11 10:00:00               Good            1260.0
179 2017-11-11 06:00:00               Good            1260.0
190 2017-11-04 08:00:00               Good            1323.0

Porcentaje de duplicados:  18.44569288389513


In [11]:
#Al tener un porcentaje de duplicados tan grande, los eliminamos.
loop_trips.drop_duplicates(inplace=True)
print('Duplicados loop_trips:\n',loop_trips.duplicated().sum()) 
print()
print(loop_trips.info())

Duplicados loop_trips:
 0

<class 'pandas.core.frame.DataFrame'>
Index: 871 entries, 0 to 1067
Data columns (total 3 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   start_ts            871 non-null    datetime64[ns]
 1   weather_conditions  871 non-null    object        
 2   duration_seconds    871 non-null    float64       
dtypes: datetime64[ns](1), float64(1), object(1)
memory usage: 27.2+ KB
None


Respecto a los ausentes, no tenemos problemas, debido a que no existen valores ausentes en ninguno de los dataframes. Por otra parte, tenemos duplicados únicamente en el dataframe loop_trips, con un porcentaje del 18% aproximadamente, por esta razón los eliminamos.

## 4. Análisis

### 4.1 Barrios destino más populares.

In [12]:
#Ordenamos los valores de mayor a menor promedio de viajes y redondeamos
hood=trips.sort_values(by='average_trips',ascending=False)
hood_top_10=hood.head(10)
hood_top_10['average_trips']=hood_top_10['average_trips'].round(2)
hood_top_10

C:\Users\Santiago\AppData\Local\Temp\ipykernel_21032\2643034947.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hood_top_10['average_trips']=hood_top_10['average_trips'].round(2)


,dropoff_location_name,average_trips
0,Loop,10727.47
1,River North,9523.67
2,Streeterville,6664.67
3,West Loop,5163.67
4,O'Hare,2546.90
5,Lake View,2420.97
6,Grant Park,2068.53
7,Museum Campus,1510.00
8,Gold Coast,1364.23
9,Sheffield & DePaul,1259.77


In [13]:
#Creamos un diagrama de barras para visualiazar mejor
fig=px.bar(hood_top_10,x='dropoff_location_name',y='average_trips',title='Top 10 barrios principales', color='dropoff_location_name')
fig.update_traces(showlegend=False)
fig.show()

Podemos ver en la anterior gráfica los 10 barrios principales, destacando a **Loop, River North y Streeterville** como los 3 más populares con **10727.47, 9523.67 y 6664.67** viajes en promedio respectivamente.

### 4.2 Número de viajes por empresas de taxi

In [14]:
#Ordenamos de mayor a menor las empresas con más viajes
taxi_company=cabs.sort_values(by='trips_amount',ascending=False)
top10_taxi_company=taxi_company.head(10)
top10_taxi_company

,company_name,trips_amount
0,Flash Cab,19558
1,Taxi Affiliation Services,11422
2,Medallion Leasin,10367
3,Yellow Cab,9888
4,Taxi Affiliation Service Yellow,9299
5,Chicago Carriage Cab Corp,9181
6,City Service,8448
7,Sun Taxi,7701
8,Star North Management LLC,7455
9,Blue Ribbon Taxi Association Inc.,5953


In [15]:
#Creamos diagrama de barras para visualizar mejor
fig1=px.bar(top10_taxi_company,x='company_name',y='trips_amount',title='Top 10 empresas de taxi principales', color='company_name')
fig1.update_traces(showlegend=False)
fig1.show()

In [16]:
print('Número total de viajes: ',cabs['trips_amount'].sum())

Número total de viajes:  137311


En la gráfica anterior podemos ver las empresas más popilares por numero de viajes realizados, de los cuales tenemos a la empresa **Flash Cab** como la que tiene más viajes con **19558 viajes**, seguido por la empresa **Taxi Afiliation Services** con **11422 viajes** y la empresa **Medallion Leasing** con **10367 viajes** en total de un total de **64 empresas y 137311 viajes**.

## 5.Conclusiones

1. En conclusión los 3 barrios más populares en cuanto a finalización de viajes son Loop, River North y Streeterville debido a que su promedio de viajes es más alto, esto se debe a que la mayoría de estos barrios y el resto de barrios de la lista quedan en el Downtown y cerga al lago michigan los cuales son las zonas turisticas más representativas de Chicago. El unico barrio que no queda cerca a la zona es O'Hare, el cual es el barrio donde está el aeropuerto, por esta razón se encuentra en la lista, al Chicago ser una de las ciudades más turisticas de Estados Unidos, recibe más viajes en sus destinos turisticos.

2. Las 3 empresas más conocidas de Chicago son Flash Cab, Taxi Afiliation Services y Medallion Leasing, los cuales son buenas opciones para invertir debido a su alta cantidad de viajes, sin embargo, sería importante hacer un análisis más detallado de estas empresas.

## 6. Prueba de hipótesis

### 6.1 Prueba de Hipotesis entre el barrio Loop y el Aeropuerto Internacional O'Hare 

"La duración promedio de los viajes desde el Loop hasta el Aeropuerto Internacional O'Hare cambia los sábados lluviosos".

H0: La duración promedio de los viajes entre Loop hasta el Aeropuerto Internacional O'hare los días lluviosos y los días soleados es igual.  
H1: La duración promedio de los viajes entre Loop hasta el Aeropuerto Internacional O'hare los días lluviosos y los días soleados **NO** es igual.  
Indice de significancia del 5%

Debido a que debemos validar si los promedios son iguales o no, dependiendo el clima y que no conocemos las desviaciones estandar de cada grupo, usaremos una prueba T-test de una cola.

In [17]:
#Revisamos el dataframe de los viajes
loop_trips.sample(10)

,start_ts,weather_conditions,duration_seconds
416,2017-11-11 09:00:00,Good,1571.0
214,2017-11-11 16:00:00,Good,2940.0
626,2017-11-04 08:00:00,Good,1260.0
342,2017-11-11 16:00:00,Good,2141.0
391,2017-11-25 12:00:00,Good,1500.0
667,2017-11-11 13:00:00,Good,2040.0
775,2017-11-18 07:00:00,Bad,1454.0
183,2017-11-11 16:00:00,Good,2293.0
442,2017-11-11 08:00:00,Good,1205.0
340,2017-11-18 08:00:00,Bad,1440.0


In [18]:
loop_trips['weather_conditions'].value_counts()

weather_conditions
Good    723
Bad     148
Name: count, dtype: int64

In [19]:
#Separamos los datos de días lluviosos y dias soleados
good_trips=loop_trips[loop_trips['weather_conditions']=='Good']['duration_seconds']
bad_trips=loop_trips[loop_trips['weather_conditions']=='Bad']['duration_seconds']
var_good=np.var(good_trips)
var_bad=np.var(bad_trips)
print(f'Varianza de los días soleados: {var_good}\nVarianza de los días lluviosos: {var_bad}')

Varianza de los días soleados: 597389.5417434273
Varianza de los días lluviosos: 561003.0958728999


In [20]:
#Hacemos la prueba de hipotesis
alpha=0.05
results=st.ttest_ind(good_trips,bad_trips)
print(f'El promedio de los viajes soleados : {good_trips.mean()}\nEl promedio de los viajes lluviosos es : {bad_trips.mean()}\n\nt-statistic: {results[0]}\np-value: {results[1]}')
if results.pvalue < alpha: # comparar el valor p con el umbral
    print("Rechazamos la hipótesis nula")
else:
    print("No podemos rechazar la hipótesis nula") 

El promedio de los viajes soleados : 2032.253112033195
El promedio de los viajes lluviosos es : 2409.2297297297296

t-statistic: -5.427957689754561
p-value: 7.397770692813658e-08
Rechazamos la hipótesis nula
